In [ ]:
"""+-----------------------------+   +-----------------------------+   +-----------------------------+
|        Process Memory       |   |        Process Memory       |   |        Process Memory       |
| (shared among all threads) |   | (shared among all threads) |   | (shared among all threads) |
+-----------------------------+   +-----------------------------+   +-----------------------------+
|    Code / Text Segment      | <--------- Shared across all threads --------->
+-----------------------------+
|        Data Segment         | <--------- Shared across all threads --------->
+-----------------------------+
|        Heap (malloc)        | <--------- Shared across all threads --------->
+-----------------------------+
|                             |
|      Kernel-managed         |
|      Thread Table &         |
|      Scheduling Info        |
+-----------------------------+
         /          |          \
        /           |           \
       v            v            v
+-------------+ +-------------+ +-------------+
|  Thread 1   | |  Thread 2   | |  Thread 3   |
+-------------+ +-------------+ +-------------+
|  Thread ID  | |  Thread ID  | |  Thread ID  |
+-------------+ +-------------+ +-------------+
|  Registers  | |  Registers  | |  Registers  |
+-------------+ +-------------+ +-------------+
| Program Ctr | | Program Ctr | | Program Ctr |
+-------------+ +-------------+ +-------------+
|   Stack     | |   Stack     | |   Stack     |
| (private)   | | (private)   | | (private)   |
+-------------+ +-------------+ +-------------+ """


'+-----------------------------+   +-----------------------------+   +-----------------------------+\n|        Process Memory       |   |        Process Memory       |   |        Process Memory       |\n| (shared among all threads) |   | (shared among all threads) |   | (shared among all threads) |\n+-----------------------------+   +-----------------------------+   +-----------------------------+\n|    Code / Text Segment      | <--------- Shared across all threads --------->\n+-----------------------------+\n|        Data Segment         | <--------- Shared across all threads --------->\n+-----------------------------+\n|        Heap (malloc)        | <--------- Shared across all threads --------->\n+-----------------------------+\n|                             |\n|      Kernel-managed         |\n|      Thread Table &         |\n|      Scheduling Info        |\n+-----------------------------+\n         /          |                  /           |                  v            v       

In [2]:
# -*- coding: utf-8 -*-
"""GIL Demystified, True Parallelism via Multiprocessing


Day 3, Session 3, Topic 1: GIL Demystified, True Parallelism via Multiprocessing

This file provides a detailed explanation of the Global Interpreter Lock (GIL)
and how it affects concurrency in Python.  It also demonstrates how to achieve
true parallelism using the `multiprocessing` module.
"""

import threading
import multiprocessing
import time

# -----------------------------------------------------------------------------
# The Global Interpreter Lock (GIL)
# -----------------------------------------------------------------------------
#
# The Global Interpreter Lock (GIL) is a mutex (a lock that protects a shared
# resource) that exists within the Python interpreter.  It allows only one
# thread to hold control of the Python interpreter at any one time.
#
# Why does the GIL exist?
#
# The GIL was introduced primarily because CPython's memory management is not
# thread-safe.  The GIL simplifies the implementation of the interpreter by
# preventing race conditions and ensuring that objects are accessed by only
# one thread at a time.  This was a design decision made early in Python's
# development to make the language easier to implement and to improve
# performance on single-threaded programs.
#
# Impact on Python's Concurrency:
#
# The GIL has a significant impact on the performance of multi-threaded
# Python programs, especially those that are CPU-bound.  While multiple threads
# can be created and managed by the operating system, only one thread can
# actually execute Python bytecode at any given time.  This effectively
# serializes the execution of Python code, preventing true parallelism on
# multi-core processors.
#
# Visual Representation:
#
# Imagine a multi-core CPU with several threads trying to execute Python code:
#
# +-------+  +-------+  +-------+  +-------+
# | Core 1|  | Core 2|  | Core 3|  | Core 4|  <- CPU Cores
# +-------+  +-------+  +-------+  +-------+
#     |          |          |          |
#     |          |          |          |
# +-------+  +-------+  +-------+  +-------+
# |Thread 1|  |Thread 2|  |Thread 3|  |Thread 4|  <- Python Threads
# +-------+  +-------+  +-------+  +-------+
#     |          |          |          |
#     |          |          |          |
#     \------[GIL]------/
#
# Only one thread can hold the GIL and execute Python code at a time.
# The other threads are blocked, even if the CPU has multiple cores available.
#
# -----------------------------------------------------------------------------
# Threads and the GIL
# -----------------------------------------------------------------------------
#
# Python's `threading` module allows you to create and manage threads.  However,
# due to the GIL, using threads for CPU-bound tasks in Python does not provide
# a significant performance improvement.  In fact, it can sometimes be slower
# due to the overhead of thread management and context switching.
#
# Example: Threading with a CPU-bound task
def cpu_bound_task(n):
    """
    A CPU-bound task that performs a long calculation.

    Args:
        n: The number of iterations.
    """
    result = 0
    for i in range(n):
        result += i * i
    return result

def demonstrate_threading_cpu_bound():
    """
    Demonstrates the effect of the GIL on CPU-bound tasks using threads.
    """
    start_time = time.time()
    threads = []
    for _ in range(4):  # Create 4 threads and run them simultaneously
        thread = threading.Thread(target=cpu_bound_task, args=(10000000,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()  # Wait for all threads to finish

    end_time = time.time()
    print(f"Threading (CPU-bound) time: {end_time - start_time:.4f} seconds")
# -----------------------------------------------------------------------------
# Processes and Multiprocessing
# -----------------------------------------------------------------------------
#
# The `multiprocessing` module in Python provides a way to create and manage
# processes.  Unlike threads, processes have their own memory space and
# Python interpreter.  Therefore, they are not affected by the GIL.
# Using multiple processes allows you to achieve true parallelism in Python,
# enabling you to fully utilize multi-core processors for CPU-bound tasks.
#
# How Multiprocessing Bypasses the GIL:
#
# Each process created with `multiprocessing` gets its own instance of the
# Python interpreter.  Since each interpreter has its own GIL, there is no
# contention for a single lock.  The operating system can schedule these
# processes to run on different CPU cores, allowing them to execute
# Python code in parallel.
#
# Example: Multiprocessing with a CPU-bound task
def demonstrate_multiprocessing_cpu_bound():
    """
    Demonstrates true parallelism for CPU-bound tasks using multiprocessing.
    """
    start_time = time.time()
    processes = []
    for _ in range(4):  # Create 4 processes
        process = multiprocessing.Process(target=cpu_bound_task, args=(10000000,))
        print(type(process))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()  # Wait for all processes to finish

    end_time = time.time()
    print(f"Multiprocessing (CPU-bound) time: {end_time - start_time:.4f} seconds")
# -----------------------------------------------------------------------------
# Inter-Process Communication (IPC)
# -----------------------------------------------------------------------------
#
# When using multiple processes, it's often necessary for them to communicate
# with each other.  The `multiprocessing` module provides several mechanisms
# for Inter-Process Communication (IPC):
#
# -   Pipes:  A simple way to create a unidirectional or bidirectional
#     communication channel between two processes.
#
# -   Queues:  A thread-safe and process-safe queue implementation that
#     allows processes to exchange data.
#
# -   Shared memory:  A way for processes to access a common region of
#     memory.
#
# -   Managers:  Provides a way to create data that can be shared between
#     processes, including lists, dictionaries, and other Python objects.
#
# Example: Inter-process communication using a queue
def demonstrate_ipc_queue():
    """
    Demonstrates inter-process communication using a queue.
    """
    def worker(q):
        """
        A worker process that puts data into the queue.

        Args:
            q: A multiprocessing.Queue object.
        """
        q.put("Hello from process")
        q.put(123)
        q.put([4, 5, 6])

    q = multiprocessing.Queue()
    p = multiprocessing.Process(target=worker, args=(q,))
    p.start()

    print("Main process receiving data...")
    print(q.get())  # Get data from the queue
    print(q.get())
    print(q.get())

    p.join()
    print("Queue Example Done")
# -----------------------------------------------------------------------------
# Detailed Comparison: Threading vs. Multiprocessing for CPU-bound Tasks
# -----------------------------------------------------------------------------
#
# To further illustrate the difference between threading and multiprocessing
# for CPU-bound tasks, let's consider a more detailed example.  We'll perform
# a series of computationally intensive calculations and measure the time it
# takes to complete them using both threading and multiprocessing.
#
def perform_calculations(n):
    """
    Performs a series of computationally intensive calculations.

    Args:
        n: The number of iterations.
    """
    result = 0
    for i in range(n):
        result += i * i
        result = result**2
        result = result % (i + 1)
    return result

def time_calculations(func, num_tasks, iterations):
    """
    Times the execution of a function with multiple tasks.

    Args:
        func: The function to execute (either with threads or processes).
        num_tasks: The number of tasks to create.
        iterations: The number of iterations for each task.
    """
    start_time = time.time()
    tasks = []
    for _ in range(num_tasks):
        if func == threading.Thread:
            task = threading.Thread(target=perform_calculations, args=(iterations,))
        elif func == multiprocessing.Process:
            task = multiprocessing.Process(target=perform_calculations, args=(iterations,))
        else:
            raise ValueError("Invalid function type")
        tasks.append(task)
        task.start()

    for task in tasks:
        task.join()

    end_time = time.time()
    return end_time - start_time

def demonstrate_threading_vs_multiprocessing_detailed():
    """
    Compares threading and multiprocessing for a detailed CPU-bound task.
    """
    num_tasks = 4
    iterations = 500000  # Increased iterations for more pronounced difference

    print("\nDetailed Comparison: Threading vs. Multiprocessing (CPU-bound)")
    threading_time = time_calculations(threading.Thread, num_tasks, iterations)
    print(f"Threading time: {threading_time:.4f} seconds")

    multiprocessing_time = time_calculations(multiprocessing.Process, num_tasks, iterations)
    print(f"Multiprocessing time: {multiprocessing_time:.4f} seconds")

    print(f"Speedup: {threading_time / multiprocessing_time:.2f}x")
# -----------------------------------------------------------------------------
# Main Execution
# -----------------------------------------------------------------------------
#
if __name__ == "__main__":
    print("Demonstrating Threading (CPU-bound):")
    demonstrate_threading_cpu_bound()
    print("\nDemonstrating Multiprocessing (CPU-bound):")
    demonstrate_multiprocessing_cpu_bound()
    print("\nDemonstrating Inter-Process Communication (Queue):")
    demonstrate_ipc_queue()
    demonstrate_threading_vs_multiprocessing_detailed()

Demonstrating Threading (CPU-bound):
Threading (CPU-bound) time: 4.1126 seconds

Demonstrating Multiprocessing (CPU-bound):
<class 'multiprocessing.context.Process'>
<class 'multiprocessing.context.Process'>
<class 'multiprocessing.context.Process'>
<class 'multiprocessing.context.Process'>
Multiprocessing (CPU-bound) time: 3.1833 seconds

Demonstrating Inter-Process Communication (Queue):
Main process receiving data...
Hello from process
123
[4, 5, 6]
Queue Example Done

Detailed Comparison: Threading vs. Multiprocessing (CPU-bound)
Threading time: 0.6958 seconds
Multiprocessing time: 0.8593 seconds
Speedup: 0.81x


In [ ]:
"""+=============================+     +=============================+     +=============================+
|      Process 1              |     |      Process 2              |     |      Process 3              |
+=============================+     +=============================+     +=============================+
|    Python Interpreter       |     |    Python Interpreter       |     |    Python Interpreter       |
|  (Separate CPython instance)|     |  (Separate CPython instance)|     |  (Separate CPython instance)|
+-----------------------------+     +-----------------------------+     +-----------------------------+
|    Code / Text Segment      |     |    Code / Text Segment      |     |    Code / Text Segment      |
+-----------------------------+     +-----------------------------+     +-----------------------------+
|        Data Segment         |     |        Data Segment         |     |        Data Segment         |
+-----------------------------+     +-----------------------------+     +-----------------------------+
|        Heap (Objects)       |     |        Heap (Objects)       |     |        Heap (Objects)       |
+-----------------------------+     +-----------------------------+     +-----------------------------+
|          Stack              |     |          Stack              |     |          Stack              |
+-----------------------------+     +-----------------------------+     +-----------------------------+
|   OS-managed Process Info   |     |   OS-managed Process Info   |     |   OS-managed Process Info   |
+-----------------------------+     +-----------------------------+     +-----------------------------+

          |                           |                             |
          |                           |                             |
          v                           v                             v

    +=============================================================+
    |              Inter-Process Communication (IPC)              |
    |    (Queue, Pipe, Socket, Shared Memory, Manager, etc.)      |
    +=============================================================+
"""